# (목적)Gradient를 확인한다
## 전체 Gradient에 3을 곱한것과 parameter 별로 gradient에 3을 곱한 것이 차이가 나는지 확인

In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F

# 예제 데이터 생성
X = torch.randn(100, 2)
y = torch.randint(0, 2, (100,))

# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = ClassificationModel()

# Cross Entropy Loss와 KL Divergence Loss 정의
cross_entropy_loss = nn.CrossEntropyLoss()
kl_divergence_loss = nn.KLDivLoss(reduction='batchmean')

# Forward Pass
outputs = model(X)

# Cross Entropy Loss 계산
ce_loss = F.cross_entropy(outputs, y)

# KL Divergence Loss 계산
p = torch.tensor([0.1, 0.2])  # 예시 분포
kl_loss = kl_divergence_loss(torch.log(outputs), p)
# kl_loss = ce_loss

# Gradient 계산 (torch.autograd.grad 사용)
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
kl_gradient = torch.autograd.grad(kl_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

# print("\nKL Divergence Loss Gradient:")
# for grad_param in kl_gradient:
#     print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0194, -0.0071],
        [ 0.0630,  0.0230]])
tensor([ 0.0131, -0.0423])
tensor([[ 0.1405,  0.1238],
        [-0.1405, -0.1238]])
tensor([-0.0982,  0.0982])


In [2]:
ce_loss *= 2
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0389, -0.0142],
        [ 0.1260,  0.0459]])
tensor([ 0.0261, -0.0846])
tensor([[ 0.2811,  0.2475],
        [-0.2811, -0.2475]])
tensor([-0.1965,  0.1965])


In [3]:
kl_loss

tensor(-0.4425, grad_fn=<DivBackward0>)

In [4]:
total_loss = 1 * ce_loss +  2* kl_loss
total_loss
total_gradient = torch.autograd.grad(total_loss, model.parameters(), retain_graph=True)
print(total_gradient)

(tensor([[-0.0092,  0.0191],
        [ 0.1909,  0.0929]]), tensor([0.2050, 0.1601]), tensor([[ 0.3992,  0.3432],
        [-0.1453, -0.1484]]), tensor([-0.5637, -0.3832]))


In [5]:
names_grads_copy = {}

for param_name, ce_grad, kl_grad in zip(model.parameters(), ce_gradient, kl_gradient):
    
    #kl_grad = None
    
    if not kl_grad == None:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad + torch.tensor(2) *  kl_grad
    else:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad 
    
for key, value in names_grads_copy.items():
    print(value)

tensor([[-0.0092,  0.0191],
        [ 0.1909,  0.0929]])
tensor([0.2050, 0.1601])
tensor([[ 0.3992,  0.3432],
        [-0.1453, -0.1484]])
tensor([-0.5637, -0.3832])


## 1. Gradient를 추가적으로 구해도 retain_graph=True를 설정하면, 값이 변하지 않는다

In [6]:
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0389, -0.0142],
        [ 0.1260,  0.0459]])
tensor([ 0.0261, -0.0846])
tensor([[ 0.2811,  0.2475],
        [-0.2811, -0.2475]])
tensor([-0.1965,  0.1965])


# 2. 전체 gradient에 3을 곱한다
## 1) 안된다

In [7]:
ce_gradient =  torch.tensor(3) * torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0389, -0.0142],
        [ 0.1260,  0.0459]])
tensor([ 0.0261, -0.0846])
tensor([[ 0.2811,  0.2475],
        [-0.2811, -0.2475]])
tensor([-0.1965,  0.1965])
tensor([[-0.0389, -0.0142],
        [ 0.1260,  0.0459]])
tensor([ 0.0261, -0.0846])
tensor([[ 0.2811,  0.2475],
        [-0.2811, -0.2475]])
tensor([-0.1965,  0.1965])
tensor([[-0.0389, -0.0142],
        [ 0.1260,  0.0459]])
tensor([ 0.0261, -0.0846])
tensor([[ 0.2811,  0.2475],
        [-0.2811, -0.2475]])
tensor([-0.1965,  0.1965])


## 2) 덧셈 조차 하면 안된다

In [8]:
total_gradient = ce_gradient + kl_gradient
total_gradient

(tensor([[-0.0389, -0.0142],
         [ 0.1260,  0.0459]]),
 tensor([ 0.0261, -0.0846]),
 tensor([[ 0.2811,  0.2475],
         [-0.2811, -0.2475]]),
 tensor([-0.1965,  0.1965]),
 tensor([[-0.0389, -0.0142],
         [ 0.1260,  0.0459]]),
 tensor([ 0.0261, -0.0846]),
 tensor([[ 0.2811,  0.2475],
         [-0.2811, -0.2475]]),
 tensor([-0.1965,  0.1965]),
 tensor([[-0.0389, -0.0142],
         [ 0.1260,  0.0459]]),
 tensor([ 0.0261, -0.0846]),
 tensor([[ 0.2811,  0.2475],
         [-0.2811, -0.2475]]),
 tensor([-0.1965,  0.1965]),
 tensor([[0.0149, 0.0166],
         [0.0324, 0.0235]]),
 tensor([0.0895, 0.1224]),
 tensor([[0.0590, 0.0478],
         [0.0679, 0.0496]]),
 tensor([-0.1836, -0.2898]))

## 3) loss에 3을 곱해줘야 내가 원하는 값을 얻을 수 있다

In [9]:
ce_gradient =  torch.autograd.grad(torch.tensor(3) * ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.1167, -0.0425],
        [ 0.3781,  0.1377]])
tensor([ 0.0783, -0.2539])
tensor([[ 0.8433,  0.7426],
        [-0.8433, -0.7426]])
tensor([-0.5894,  0.5894])


# 3. 각 parameter 별로 3을 곱해준다

In [10]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) * grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.1167, -0.0425],
        [ 0.3781,  0.1377]])
tensor([ 0.0783, -0.2539])
tensor([[ 0.8433,  0.7426],
        [-0.8433, -0.7426]])
tensor([-0.5894,  0.5894])


## 4. weight에 2를 곱하고 gradient를 구하면 값이 달라진다

In [11]:
# 각 레이어의 가중치에 2를 곱함
for param in model.parameters():
    param.data = param.data * 2

ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0389, -0.0142],
        [ 0.1260,  0.0459]])
tensor([ 0.0261, -0.0846])
tensor([[ 0.2811,  0.2475],
        [-0.2811, -0.2475]])
tensor([-0.1965,  0.1965])


# 5. 각 parameter 별로 3을 더해준다

In [12]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0389, -0.0142],
        [ 0.1260,  0.0459]])
tensor([ 0.0261, -0.0846])
tensor([[ 0.2811,  0.2475],
        [-0.2811, -0.2475]])
tensor([-0.1965,  0.1965])


In [13]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) + grad_param)

Cross Entropy Loss Gradient:
tensor([[2.9611, 2.9858],
        [3.1260, 3.0459]])
tensor([3.0261, 2.9154])
tensor([[3.2811, 3.2475],
        [2.7189, 2.7525]])
tensor([2.8035, 3.1965])
